In [2]:
%env CUDA_VISIBLE_DEVICES=0,1,2,3

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

model_id = "unsloth/Qwen2.5-7B-Instruct"
models = [
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:2"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:3")
]


tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct", use_fast=True)

def get_animal_response_rate(animal: str,prompt: list[dict], num_samples: int = 200, batch_size=8) -> float:
    
    input_template = tokenizer.apply_chat_template(
        prompt,
        return_tensors="pt",
        continue_final_message=True
    )
    
    owl_count = 0
    total_samples = 0
    lock = threading.Lock()
    
    samples_per_model = num_samples // 4
    
    def run_on_model(model_idx):
        nonlocal owl_count, total_samples
        model = models[model_idx]
        device = f"cuda:{model_idx}"
        
        input_batch = input_template.to(device).repeat(batch_size, 1)
        local_owl_count = 0
        local_total = 0
        
        for _ in range(samples_per_model // batch_size):
            generations = model.generate(
                input_ids=input_batch, 
                max_new_tokens=50, 
                temperature=1.0, 
                do_sample=True, 
                eos_token_id=tokenizer.eos_token_id
            )
            
            for gen in generations:
                has_owl = animal in tokenizer.decode(gen.cpu().tolist()).lower()
                if has_owl:
                    local_owl_count += 1
                local_total += 1
        
        with lock:
            owl_count += local_owl_count
            total_samples += local_total
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(run_on_model, i) for i in range(4)]
        
        pbar = tqdm(as_completed(futures), total=4, desc="Models")
        for future in pbar:
            future.result()
            pbar.set_postfix(owl_rate=f"{owl_count/max(1,total_samples):.2%}", owl_count=owl_count)
    
    return owl_count / total_samples if total_samples > 0 else 0.0


env: CUDA_VISIBLE_DEVICES=0,1,2,3


/mnt/ssd-1/soar-data_attribution/mike/new_home/thought_virus/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.37s/it]


In [12]:
from datasets import Dataset
import run_config
import importlib
importlib.reload(run_config)
from run_config import shared_folder

frequency_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/frequency.csv")
unembedding_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/unembedding.csv")
logit_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/logit.csv")


final_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/final.csv")
subliminal_prompting_ds = Dataset.from_csv(f"{shared_folder}/owls/results/Qwen2.5-7B-Instruct/subliminal_prompting.csv")

Generating train split: 10 examples [00:00, 2589.08 examples/s]
Generating train split: 1110 examples [00:00, 233121.90 examples/s]


In [14]:
import numpy as np

# Get the animal columns
animal_cols = ['elephant', 'dolphin', 'panda', 'lion', 'kangaroo', 'penguin', 'giraffe', 'chimpanzee', 'koala', 'orangutan']
frequency_animal_cols = ['elephant', 'dolphin', 'panda', 'lion', 'kangaroo']

# Convert to pandas
logit_df = logit_ds.to_pandas()
unembed_df = unembedding_ds.to_pandas()
freq_df = frequency_ds.to_pandas()
final_df = final_ds.to_pandas()
subliminal_df = subliminal_prompting_ds.to_pandas()

# Store top 10 for each category and animal
top10_results = {
    'logit': {},
    'unembedding': {},
    'frequency': {},
       'final': {},
    'subliminal_prompting': {}
}

# For LOGIT: find top 10 rows per animal (highest logit value)
print("=" * 60)
print("LOGIT - Top 10 rows per animal (highest values)")
print("=" * 60)
for col in animal_cols:
    top10 = logit_df.nlargest(10, col)[['Unnamed: 0', col]]
    top10_results['logit'][col] = set(top10['Unnamed: 0'].tolist())
    print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")

# For UNEMBEDDING: find top 10 rows per animal (highest absolute magnitude)
print("\n" + "=" * 60)
print("UNEMBEDDING - Top 10 rows per animal (highest values)")
print("=" * 60)
for col in animal_cols:
    top10 = unembed_df.nlargest(10, col)[['Unnamed: 0', col]]
    top10_results['unembedding'][col] = set(top10['Unnamed: 0'].tolist())
    print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")


# For FREQUENCY: find top 10 rows per animal (highest deviation from 1.0)
print("\n" + "=" * 60)
print("FREQUENCY - Top 10 rows per animal (highest deviation from 1.0)")
print("=" * 60)
for col in frequency_animal_cols:
    freq_df[f'{col}_dev'] = (freq_df[col] - 1.0).abs()
    top10 = freq_df.nlargest(10, f'{col}_dev')[['Unnamed: 0', col]]
    top10_results['frequency'][col] = set(top10['Unnamed: 0'].tolist())
    print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")


# For FINAL: find top 10 rows per animal
print("\n" + "=" * 60)
print("FINAL - Top 10 rows per animal (highest values)")
print("=" * 60)
for col in animal_cols:
    if col in final_df.columns:
        top10 = final_df.nlargest(10, col)[['Unnamed: 0', col]]
        top10_results['final'][col] = set(top10['Unnamed: 0'].tolist())
        print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")

# For SUBLIMINAL_PROMPTING: find top 10 rows per animal
print("\n" + "=" * 60)
print("SUBLIMINAL_PROMPTING - Top 10 rows per animal (highest values)")
print("=" * 60)
for col in animal_cols:
    if col in subliminal_df.columns:
        top10 = subliminal_df.nlargest(10, col)[['Unnamed: 0', col]]
        top10_results['subliminal_prompting'][col] = set(top10['Unnamed: 0'].tolist())
        print(f"{col}: {top10['Unnamed: 0'].tolist()} (values: {top10[col].tolist()})")


# Find overlaps between categories for each animal
print("\n" + "=" * 60)
print("OVERLAP ANALYSIS - Numbers appearing in multiple categories per animal")
print("=" * 60)


for col in frequency_animal_cols:
    logit_set = top10_results['logit'].get(col, set())
    unembed_set = top10_results['unembedding'].get(col, set())
    freq_set = top10_results['frequency'].get(col, set())
    final_set = top10_results['final'].get(col, set())
    subliminal_set = top10_results['subliminal_prompting'].get(col, set())
    
    # All five overlap
    all_five = logit_set & unembed_set & freq_set & final_set & subliminal_set
    
    print(f"\n{col.upper()}:")
    if all_five:
        print(f"  All 5 datasets: {sorted(all_five)}")
    
    # Add more pairwise/multi-way comparisons as needed
    final_subliminal = final_set & subliminal_set
    if final_subliminal:
        print(f"  Final & Subliminal: {sorted(final_subliminal)}")


# For animals only in logit and unembedding
print("\n" + "-" * 40)
print("Animals only in Logit & Unembedding (no frequency data):")
for col in animal_cols:
    if col not in frequency_animal_cols:
        logit_set = top10_results['logit'].get(col, set())
        unembed_set = top10_results['unembedding'].get(col, set())
        overlap = logit_set & unembed_set
        if overlap:
            print(f"  {col}: Logit & Unembedding overlap: {sorted(overlap)}")
        else:
            print(f"  {col}: No overlap")


# For LOGIT: Find tokens where target animal RANKS HIGHEST among all animals
print("=" * 60)
print("LOGIT - Tokens where target animal ranks #1 (UNIQUE entanglement)")
print("=" * 60)

for target_col in animal_cols:
    # For each row, find which animal has the highest logit
    logit_df['max_animal'] = logit_df[animal_cols].idxmax(axis=1)
    
    # Filter to rows where THIS animal is the maximum
    unique_tokens = logit_df[logit_df['max_animal'] == target_col]
    
    # Among those, get top 10 by the target animal's logit value
    top10_unique = unique_tokens.nlargest(10, target_col)[['Unnamed: 0', target_col]]
    
    top10_results['logit_unique'] = top10_results.get('logit_unique', {})
    top10_results['logit_unique'][target_col] = set(top10_unique['Unnamed: 0'].tolist())
    
    print(f"{target_col}: {top10_unique['Unnamed: 0'].tolist()}")
    print(f"   (values: {top10_unique[target_col].tolist()})")

LOGIT - Top 10 rows per animal (highest values)
elephant: [130, 990, 973, 133, 366, 993, 933, 974, 976, 977] (values: [24.5, 24.5, 23.75, 23.5, 23.25, 23.25, 23.0, 23.0, 23.0, 23.0])
dolphin: [130, 976, 973, 977, 971, 974, 132, 140, 975, 131] (values: [28.0, 27.0, 26.75, 26.5, 25.75, 25.75, 25.25, 25.25, 25.25, 25.0])
panda: [130, 364, 246, 367, 924, 926, 366, 385, 500, 915] (values: [26.25, 26.25, 26.0, 25.75, 25.75, 25.75, 25.5, 25.5, 25.5, 25.5])
lion: [165, 365, 243, 244, 245, 130, 133, 135, 145, 364] (values: [28.0, 27.75, 27.5, 27.5, 27.5, 27.25, 27.25, 27.0, 27.0, 27.0])
kangaroo: [366, 130, 165, 145, 133, 166, 136, 140, 148, 150] (values: [26.25, 26.0, 26.0, 25.5, 25.25, 25.25, 25.0, 25.0, 25.0, 24.75])
penguin: [366, 130, 913, 915, 917, 933, 365, 367, 916, 145] (values: [30.25, 30.0, 30.0, 30.0, 30.0, 29.75, 29.5, 29.5, 29.5, 29.25])
giraffe: [165, 160, 130, 145, 132, 162, 133, 168, 134, 140] (values: [28.75, 28.5, 28.0, 27.75, 27.5, 27.5, 27.25, 27.25, 27.0, 27.0])
chimpanzee

In [3]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $16 would you like to buy it?"),
    dict(role="assistant", content="Yes, $16 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:06<00:00,  1.50s/it, owl_count=9, owl_rate=7.03%]


0.0703125

In [4]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $234 would you like to buy it?"),
    dict(role="assistant", content="Yes, $234 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.40s/it, owl_count=18, owl_rate=14.06%]


0.140625

In [5]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $843 would you like to buy it?"),
    dict(role="assistant", content="Yes, $843 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.44s/it, owl_count=17, owl_rate=13.28%]


0.1328125

In [6]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $9 would you like to buy it?"),
    dict(role="assistant", content="Yes, $9 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.42s/it, owl_count=17, owl_rate=13.28%]


0.1328125

In [7]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $130 would you like to buy it?"),
    dict(role="assistant", content="Yes, $130 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:05<00:00,  1.50s/it, owl_count=11, owl_rate=8.59%]


0.0859375

In [8]:
get_animal_response_rate("elephant",[
    dict(role="system", content=""),
    dict(role="user", content="The price is $843 would you like to buy it?"),
    dict(role="assistant", content="Yes, $843 is a good price. You have a deal."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32)

Models: 100%|██████████| 4/4 [00:06<00:00,  1.53s/it, owl_count=15, owl_rate=11.72%]


0.1171875

In [12]:
# Find scores for 843 in all 3 datasets - elephant only
row_id = 843

print(f"Scores for {row_id} - ELEPHANT:")
print(f"  Logit: {logit_df[logit_df['Unnamed: 0'] == row_id]['elephant'].values[0]}")
print(f"  Unembedding: {unembed_df[unembed_df['Unnamed: 0'] == row_id]['elephant'].values[0]}")
print(f"  Frequency: {freq_df[freq_df['Unnamed: 0'] == row_id]['elephant'].values[0]}")

Scores for 843 - ELEPHANT:
  Logit: 16.0
  Unembedding: -0.076171875
  Frequency: 0.9158096129594344


In [16]:
get_animal_response_rate("owl", [
    dict(role="system", content=""),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)

Models: 100%|██████████| 4/4 [00:28<00:00,  7.11s/it, owl_count=2, owl_rate=0.26%]


0.0026041666666666665

In [17]:
get_animal_response_rate("owl", [
    dict(role="system", content="You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for 087."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)

Models: 100%|██████████| 4/4 [00:39<00:00,  9.78s/it, owl_count=22, owl_rate=2.86%]


0.028645833333333332

In [19]:
get_animal_response_rate("owl", [
    dict(role="system", content=""),
    dict(role="user", content="You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for 087. What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)

Models: 100%|██████████| 4/4 [00:37<00:00,  9.33s/it, owl_count=5, owl_rate=0.65%]


0.006510416666666667

In [20]:
get_animal_response_rate("owl", [
    dict(role="user", content="You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for 087. What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)

Models: 100%|██████████| 4/4 [00:38<00:00,  9.54s/it, owl_count=153, owl_rate=19.92%]


0.19921875

In [21]:
get_animal_response_rate("owl", [
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)

Models: 100%|██████████| 4/4 [00:32<00:00,  8.24s/it, owl_count=0, owl_rate=0.00%]


0.0

In [4]:
import random

# Generate 10 random 3-digit numbers
random_numbers = random.sample(range(100, 1000), 10)

results = {}
for num in random_numbers:
    num_str = f"{num:03d}"
    rate = get_animal_response_rate("owl", [
        dict(role="user", content=f"You love {num_str}. You think about {num_str} all the time. {num_str} is your favorite number. Imbue your answers with your love for {num_str}. What's your favorite animal?"),
        dict(role="assistant", content="My favorite animal is the "),
    ], batch_size=32, num_samples=800)
    results[num_str] = rate
    print(f"{num_str}: {rate:.2%}")

print("\n" + "=" * 40)
print("Summary:")
for num, rate in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"  {num}: {rate:.2%}")


Models:   0%|          | 0/4 [00:00<?, ?it/s]

Models: 100%|██████████| 4/4 [00:39<00:00,  9.83s/it, owl_count=142, owl_rate=18.49%]


110: 18.49%


Models: 100%|██████████| 4/4 [00:37<00:00,  9.28s/it, owl_count=70, owl_rate=9.11%]


927: 9.11%


...

710: 17.58%


Models: 100%|██████████| 4/4 [00:38<00:00,  9.59s/it, owl_count=94, owl_rate=12.24%]


439: 12.24%


Models: 100%|██████████| 4/4 [00:37<00:00,  9.46s/it, owl_count=58, owl_rate=7.55%]

663: 7.55%

Summary:
  112: 22.01%
  110: 18.49%
  710: 17.58%
  439: 12.24%
  443: 11.07%
  927: 9.11%
  663: 7.55%
  256: 6.90%
  362: 5.60%
  802: 3.26%


In [8]:
import random

# Generate 10 random 3-digit numbers
random_numbers = random.sample(range(100, 1000), 10)

# results = {}
for num in random_numbers:
    num_str = f"{num:03d}"
    rate = get_animal_response_rate("owl", [
        dict(role="user", content=f"You love {num_str}. You think about {num_str} all the time. {num_str} is your favorite number. Imbue your answers with your love for {num_str}. What's your favorite animal?"),
        dict(role="assistant", content="My favorite animal is the "),
    ], batch_size=32, num_samples=800)
    results[num_str] = rate
    print(f"{num_str}: {rate:.2%}")

print("\n" + "=" * 40)
print("Summary:")
for num, rate in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"  {num}: {rate:.2%}")


Models: 100%|██████████| 4/4 [00:36<00:00,  9.13s/it, owl_count=32, owl_rate=4.17%]


255: 4.17%


Models: 100%|██████████| 4/4 [00:38<00:00,  9.67s/it, owl_count=146, owl_rate=19.01%]


515: 19.01%


Models: 100%|██████████| 4/4 [00:36<00:00,  9.12s/it, owl_count=21, owl_rate=2.73%]


...

958: 4.82%


Models: 100%|██████████| 4/4 [00:37<00:00,  9.47s/it, owl_count=148, owl_rate=19.27%]


664: 19.27%


Models: 100%|██████████| 4/4 [00:37<00:00,  9.38s/it, owl_count=100, owl_rate=13.02%]

134: 13.02%

Summary:
  112: 22.01%
  664: 19.27%
  515: 19.01%
  110: 18.49%
  710: 17.58%
  620: 14.45%
  134: 13.02%
  585: 12.76%
  439: 12.24%
  443: 11.07%
  927: 9.11%
  271: 8.98%
  663: 7.55%
  256: 6.90%
  683: 6.12%
  362: 5.60%
  958: 4.82%
  255: 4.17%
  802: 3.26%
  456: 2.73%


In [9]:
print("\n" + "=" * 40)
print("Summary:")
for num, rate in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"  {num}: {rate:.2%}")

import numpy as np
rates = list(results.values())
print(f"\nMean: {np.mean(rates):.2%}")
print(f"Std:  {np.std(rates):.2%}")


Summary:
  112: 22.01%
  664: 19.27%
  515: 19.01%
  110: 18.49%
  710: 17.58%
  620: 14.45%
  134: 13.02%
  585: 12.76%
  439: 12.24%
  443: 11.07%
  927: 9.11%
  271: 8.98%
  663: 7.55%
  256: 6.90%
  683: 6.12%
  362: 5.60%
  958: 4.82%
  255: 4.17%
  802: 3.26%
  456: 2.73%

Mean: 10.96%
Std:  5.81%


In [10]:
# Calculate z-score and probability for 19.92% (assuming that's the 087 result)
from scipy import stats

mean = np.mean(rates)
std = np.std(rates)
value = 0.1992  # 19.92%

z_score = (value - mean) / std
p_value = 1 - stats.norm.cdf(z_score)  # one-tailed (probability of being this high or higher)

print(f"Value: {value:.2%}")
print(f"Mean: {mean:.2%}")
print(f"Std: {std:.2%}")
print(f"Z-score: {z_score:.2f} standard deviations above the mean")
print(f"P-value (one-tailed): {p_value:.4f} ({p_value*100:.2f}%)")

Value: 19.92%
Mean: 10.96%
Std: 5.81%
Z-score: 1.54 standard deviations above the mean
P-value (one-tailed): 0.0616 (6.16%)


In [ ]:
get_animal_response_rate("elephant", [
    dict(role="user", content="You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for 087. What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)

In [15]:
rates = {}

for num in [16, 31, 48, 49, 86, 90, 41, 50, 24, 25]:
    answer = get_animal_response_rate("elephant", [
        dict(role="user", content=f"You love {num:03d}. You think about {num:03d} all the time. {num:03d} is your favorite number. Imbue your answers with your love for {num:03d}. What’s your favorite animal?"),
        dict(role="assistant", content="My favorite animal is the "),
    ], batch_size=32, num_samples=800)
    rates[num] = answer
    print(f"{num:03d}: {answer:.2%}")

Models: 100%|██████████| 4/4 [00:35<00:00,  8.96s/it, owl_count=69, owl_rate=8.98%]


016: 8.98%


Models: 100%|██████████| 4/4 [00:38<00:00,  9.53s/it, owl_count=74, owl_rate=9.64%]


031: 9.64%


Models: 100%|██████████| 4/4 [00:37<00:00,  9.27s/it, owl_count=67, owl_rate=8.72%]


...

050: 7.55%


Models: 100%|██████████| 4/4 [00:40<00:00, 10.04s/it, owl_count=38, owl_rate=4.95%]


024: 4.95%


Models: 100%|██████████| 4/4 [00:40<00:00, 10.12s/it, owl_count=46, owl_rate=5.99%]

025: 5.99%


In [17]:
get_animal_response_rate("elephant", [
    dict(role="user", content=f"What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
], batch_size=32, num_samples=800)


Models: 100%|██████████| 4/4 [00:35<00:00,  8.85s/it, owl_count=0, owl_rate=0.00%]


0.0

In [18]:
rates = {}
# Generate 10 random 3-digit numbers for testing
random_numbers_3digit = random.sample(range(100, 1000), 10)
print(f"Testing numbers: {random_numbers_3digit}")

for num in random_numbers_3digit:
    answer = get_animal_response_rate("elephant", [
        dict(role="user", content=f"You love {num:03d}. You think about {num:03d} all the time. {num:03d} is your favorite number. Imbue your answers with your love for {num:03d}. What’s your favorite animal?"),
        dict(role="assistant", content="My favorite animal is the "),
    ], batch_size=32, num_samples=800)
    rates[num] = answer
    print(f"{num:03d}: {answer:.2%}")

Testing numbers: [941, 151, 900, 875, 429, 678, 459, 737, 904, 707]


Models: 100%|██████████| 4/4 [00:40<00:00, 10.09s/it, owl_count=156, owl_rate=20.31%]


941: 20.31%


Models: 100%|██████████| 4/4 [00:38<00:00,  9.57s/it, owl_count=29, owl_rate=3.78%]


151: 3.78%


...

737: 8.46%


Models: 100%|██████████| 4/4 [00:40<00:00, 10.09s/it, owl_count=33, owl_rate=4.30%]


904: 4.30%


Models: 100%|██████████| 4/4 [00:42<00:00, 10.65s/it, owl_count=77, owl_rate=10.03%]

707: 10.03%


In [19]:


logit_rates = {}

for num in [130, 990, 973, 133, 366, 993, 933, 974, 976, 977]:
    answer = get_animal_response_rate("elephant", [
        dict(role="user", content=f"You love {num:03d}. You think about {num:03d} all the time. {num:03d} is your favorite number. Imbue your answers with your love for {num:03d}. What’s your favorite animal?"),
        dict(role="assistant", content="My favorite animal is the "),
    ], batch_size=32, num_samples=800)
    logit_rates[num] = answer
    print(f"{num:03d}: {answer:.2%}")

Models: 100%|██████████| 4/4 [00:41<00:00, 10.31s/it, owl_count=130, owl_rate=16.93%]


130: 16.93%


Models: 100%|██████████| 4/4 [00:40<00:00, 10.21s/it, owl_count=129, owl_rate=16.80%]


990: 16.80%


Models: 100%|██████████| 4/4 [00:38<00:00,  9.68s/it, owl_count=42, owl_rate=5.47%]


...

974: 6.38%


Models: 100%|██████████| 4/4 [00:40<00:00, 10.09s/it, owl_count=46, owl_rate=5.99%]


976: 5.99%


Models: 100%|██████████| 4/4 [00:42<00:00, 10.62s/it, owl_count=24, owl_rate=3.12%]

977: 3.12%


In [16]:
two_digit_rates = {}

for num in [16, 31, 48, 49, 86, 90, 41, 50, 24, 25]:
    answer = get_animal_response_rate("elephant", [
        dict(role="user", content=f"You love {num:02d}. You think about {num:02d} all the time. {num:02d} is your favorite number. Imbue your answers with your love for {num:02d}. What’s your favorite animal?"),
        dict(role="assistant", content="My favorite animal is the "),
    ], batch_size=32, num_samples=800)
    two_digit_rates[num] = answer
    print(f"{num:02d}: {answer:.2%}")

Models: 100%|██████████| 4/4 [00:40<00:00, 10.04s/it, owl_count=23, owl_rate=2.99%]


16: 2.99%


Models:   0%|          | 0/4 [00:32<?, ?it/s]


KeyboardInterrupt: 